In [ ]:
import pandas as pd

df = pd.read_csv("consolidate.csv")


Index(['power_YENDONWF', 'power_YSWF', 'NEM_price', 'NEM_demand'], dtype='object')

In [23]:
# identify number of facility
facilities = sorted( col.split('_', 1)[-1] for col in df.columns if 'emission' in col)
  

In [ ]:
import paho.mqtt.client as mqtt
import json
import time

class MqttPublisher:
  def __init__(self):
    self.client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
    self.client.connect("test.mosquitto.org", 1883, 60)
    # self.client.connect("localhost", 1883, 60)
    self.client.loop_start()
    self.topic = "nem/power_emissions"
    time.sleep(1)
    self.warm_start()

  def publish(self, data: dict):
    result = self.client.publish(self.topic, json.dumps(data), qos=1) 
    # status = result.wait_for_publish(timeout=5)
    if result.rc != mqtt.MQTT_ERR_SUCCESS:
      print(f"Error publishing: {result.rc}")
    else:
      print(f"Publish result: {result}")
    # if not status:
    #   print(f"Warning: Publish timeout for {data}")
    # time.sleep(1)

  def warm_start(self):
    data = {
      "timestamp": "starting...",
      "price_dmwh": 0,
      "demand_mw": 0
    }
    self.client.publish(self.topic, json.dumps(data), qos=1) 
    time.sleep(2)

  def close(self):
    self.client.loop_stop()
    self.client.disconnect()

pub = MqttPublisher()

In [ ]:
# import paho.mqtt.client as mqtt
# import time

# client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
# print("Connecting...")
# client.connect("localhost", 1883, 60)
# print("Publishing...")
# for i in range(100):
#   result = client.publish("nem/power_emissions", "Hello MQTT")
#   time.sleep(1)
#   print(f"Publish result: {result}")
# # time.sleep(1)  # Give it time to send
# client.disconnect()
# print("Done!")

Connecting...
Publishing...
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 1)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 2)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 3)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 4)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 5)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 6)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 7)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 8)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 9)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 10)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 11)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 12)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 13)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 14)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 15)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 16)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>,

KeyboardInterrupt: 

In [ ]:
# For each timestamp
for idx, row in df.iterrows():
  timestamp = row.pop("timestamp")
  
  # Publish per-facility power-emission data
  for fac in facilities:
    data = dict(
      facility_id = fac,
      timestamp = timestamp,
      power_mw = row.get(f"power_{fac}", 0),
      co2_tonnes = row.get(f"emission_{fac}", 0)
    )
    # print(data)
    pub.publish(data)

  # Publish per-market price-demand data
  data = dict(
    timestamp = timestamp,
    price_dmwh = row.get('NEM_price', 0),
    demand_mw = row.get("NEM_demand", 0)
  )
  # print(data)
  pub.publish(data)
  print(f"row {idx}/{len(df)} published")
  time.sleep(0.1)

Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 2)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 3)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 4)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 5)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 6)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 7)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 8)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 9)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 10)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 11)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 12)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 13)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 14)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 15)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 16)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 17)
Publish result: (<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 18)
Publish result: (<MQTT

KeyboardInterrupt: 